In [1]:
import os
import glob
import numpy as np
import bokeh.io
import bokeh.plotting
import bokeh.palettes
from bokeh.transform import jitter
import seaborn as sns
import matplotlib
from bokeh.models import HoverTool
from scipy import stats
import pandas as pd
import random

from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.transform import dodge

import matplotlib
print('matplotlib: '+matplotlib.__version__)
import matplotlib.pyplot as plt
%matplotlib inline

from bokeh.layouts import row
bokeh.io.output_notebook()

matplotlib: 3.3.2


Loading BokehJS ...

In [2]:
def persentage_difference(df_in, nb_pili_threshold):
    df_raw=df_in
    df_raw['TotalPili']=df_raw['Nb_Pili_PoleBright']+df_raw['Nb_Pili_PoleDim']
    df=df_raw.loc[df_raw['TotalPili']>nb_pili_threshold]
    NPili_DimPole=np.array(df.Nb_Pili_PoleDim)
    NPili_BrightPole=np.array(df.Nb_Pili_PoleBright)
    raw_fluo_diff=(np.array(df.TotalFluorescencePoleBright)-np.array(df.TotalFluorescencePoleDim))#/np.array(df.CellTotalFluorescence)
    raw_pili_nb_diff=(NPili_BrightPole-NPili_DimPole)
    pili_nb_diff=[]
    fluo_diff=[]
    cells_morePiliBright=[]
    cells_morePiliDim=[]                                                     
    cells_samePili=[]
    for i in range(len(NPili_DimPole)):
        #print(str(NPili_BrightPole[i]!=0) +" and "+ str(NPili_DimPole[i] != 0) + " makes " + str((NPili_BrightPole[i]!=0 or NPili_DimPole[i] != 0)))
        if(NPili_BrightPole[i]!=0 or NPili_DimPole[i] != 0):
            pili_nb_diff.append(NPili_BrightPole[i]-NPili_DimPole[i])
            fluo_diff.append(raw_fluo_diff[i])
            if(NPili_BrightPole[i] > NPili_DimPole[i]):
                cells_morePiliBright.append(1)
                cells_morePiliDim.append(0)
                cells_samePili.append(0)
            elif (NPili_BrightPole[i] < NPili_DimPole[i]):
                cells_morePiliBright.append(0)
                cells_morePiliDim.append(1)
                cells_samePili.append(0)
            else:
                cells_morePiliBright.append(0)
                cells_morePiliDim.append(0)
                cells_samePili.append(1)
    N_cells_morePiliBright=sum(cells_morePiliBright)
    N_cells_morePiliDim=sum(cells_morePiliDim)
    N_cells_samePili=sum(cells_samePili)
    #n=len(NPili_DimPole)
    n=len(cells_morePiliBright)
    persentage_piliBright=N_cells_morePiliBright/n
    persentage_piliDim=N_cells_morePiliDim/n
    persentage_samepili=N_cells_samePili/n
    return persentage_piliBright, persentage_piliDim, persentage_samepili, N_cells_morePiliBright, N_cells_morePiliDim, N_cells_samePili, n, pili_nb_diff, raw_fluo_diff

def persentage_difference_biorep(df_in, nb_pili_threshold):
    df_raw=df_in
    df_raw['TotalPili']=df_raw['Nb_Pili_PoleBright']+df_raw['Nb_Pili_PoleDim']
    df_temp=df_raw.loc[df_raw['TotalPili']>nb_pili_threshold]
    biorep=df_temp.BiologicalReplicate.unique()
    N_cells_morePiliBright=[None]*len(biorep)
    N_cells_morePiliDim=[None]*len(biorep)
    N_cells_samePili=[None]*len(biorep)
    #n=len(NPili_DimPole)
    n=[None]*len(biorep)
    persentage_piliBright=[None]*len(biorep)
    persentage_piliDim=[None]*len(biorep)
    persentage_samepili=[None]*len(biorep)
    for br in biorep:
        df=df_temp.loc[df_raw['BiologicalReplicate']== br]
        NPili_DimPole=np.array(df.Nb_Pili_PoleDim)
        NPili_BrightPole=np.array(df.Nb_Pili_PoleBright)
        pili_nb_diff=[]
        fluo_diff=[]
        cells_morePiliBright=[]
        cells_morePiliDim=[]                                                     
        cells_samePili=[]
        for i in range(len(NPili_DimPole)):
            if(NPili_BrightPole[i]!=0 or NPili_DimPole[i] != 0):
                if(NPili_BrightPole[i] > NPili_DimPole[i]):
                    cells_morePiliBright.append(1)
                    cells_morePiliDim.append(0)
                    cells_samePili.append(0)
                elif (NPili_BrightPole[i] < NPili_DimPole[i]):
                    cells_morePiliBright.append(0)
                    cells_morePiliDim.append(1)
                    cells_samePili.append(0)
                else:
                    cells_morePiliBright.append(0)
                    cells_morePiliDim.append(0)
                    cells_samePili.append(1)
        N_cells_morePiliBright[br-1]=sum(cells_morePiliBright)
        N_cells_morePiliDim[br-1]=sum(cells_morePiliDim)
        N_cells_samePili[br-1]=sum(cells_samePili)
        n[br-1]=len(cells_morePiliBright)
        persentage_piliBright[br-1]=N_cells_morePiliBright[br-1]/n[br-1]
        persentage_piliDim[br-1]=N_cells_morePiliDim[br-1]/n[br-1]
        persentage_samepili[br-1]=N_cells_samePili[br-1]/n[br-1]
    return persentage_piliBright, persentage_piliDim, persentage_samepili, N_cells_morePiliBright, N_cells_morePiliDim, N_cells_samePili, n

def persentage_bipolar_pili(df):
    a=np.array(df.Nb_Pili_PoleDim)
    b=np.array(df.Nb_Pili_PoleBright)
    pili_nb_diff=(b-a)
    c=[]
    for i in range(len(a)):
        if(b[i]!=0 or a[i] != 0):
            if(b[i] > 0 and a[i]>0):
                c.append(1)
            else:
                c.append(0)        
    d=sum(c)
    persentage_piliBright=d/len(c)
    return persentage_piliBright

def errobar(piliDim, piliDim_std, p, x_loc):
    for i in range(len(piliDim)):
        pos=x_loc+i
        p.line(
            x=[pos,pos],
            y=[piliDim[i]-piliDim_std[i], piliDim[i]+piliDim_std[i]], 
            line_color = 'black',
            alpha=1
        )

        p.line(
            x=[pos-0.02,pos+0.02],
            y=[piliDim[i]+piliDim_std[i], piliDim[i]+piliDim_std[i]],
            line_color = 'black',
            alpha=1
        )
        p.line(
            x=[pos-0.02,pos+0.02],
            y=[piliDim[i]-piliDim_std[i], piliDim[i]-piliDim_std[i]],
            line_color = 'black',
            alpha=1
        )

def plot_mean(piliDim, p, x_loc):
    for i in range(len(piliDim)):
        pos=x_loc+i
        p.circle(
            x=pos,
            y=piliDim[i], 
            line_color = 'black',
            line_width = 2,
            fill_color = 'white',
            size = 10,
            alpha=1
        )
        
def plot_data(piliDim, p, x_loc, color,jitter_width, legends):
    for i in range(len(piliDim)):
        pos=x_loc+i
        for j in range(len(piliDim[i])):
            p.circle(
                x=pos+random.uniform(-jitter_width, jitter_width),
                y=piliDim[i][j], 
                line_color = 'black',
                fill_color = color,
                size = 6,
                legend_label = legends,
                alpha=0.8
            )

In [11]:
data_path="C:/users/Iscia/Downloads/4-Mechanosensation/"
df_pilB = pd.read_csv(os.path.join(data_path, "pilB", "Combined_Data", "AllBioReps.csv"), sep=',', na_values='*')
df_pilT = pd.read_csv(os.path.join(data_path, "pilT", "Combined_Data", "AllBioReps.csv"), sep=',', na_values='*')
df_pilU = pd.read_csv(os.path.join(data_path, "pilU", "Combined_Data", "AllBioReps.csv"), sep=',', na_values='*')
df_pilG = pd.read_csv('C:/users/lorenzo/Desktop/git/PhD_codes/Mechanosensation/Python_code/Pole_analysis/mNG_pilG_fliC-/PolesData_mNG_pilG_fliC-.csv', sep=',', na_values='*')

In [12]:
persent_pilB_b, persent_pilB_d, persent_pilB_s, n_b, n_d, n_s, n_B, diff_pilB, fluo_diff_pilB = persentage_difference(df_pilB,0)
print("PilB")
print("Percent of cells with more pili at bright pole = "+str(persent_pilB_b))
print("Percent of cells with more pili at dim pole = "+str(persent_pilB_d))
print("Percent of cells with same number of pili at both poles = "+str(persent_pilB_s))
print('N cells = '+str(n_B)+',  N more at bright pole = '+str(n_b)+',  N more at dim pole = '+str(n_d)+',  N same at both pole = '+str(n_s)+'\n')

persent_pilT_b, persent_pilT_d, persent_pilT_s, n_b, n_d, n_s, n_T, diff_pilT, fluo_diff_pilT = persentage_difference(df_pilT,0)
print("PilT")
print("Percent of cells with more pili at bright pole = "+str(persent_pilT_b))
print("Percent of cells with more pili at dim pole = "+str(persent_pilT_d))
print("Percent of cells with same number of pili at both poles = "+str(persent_pilT_s))
print('N cells = '+str(n_T)+',  N more at bright pole = '+str(n_b)+',  N more at dim pole = '+str(n_d)+',  N same at both pole = '+str(n_s)+'\n')

persent_pilU_b, persent_pilU_d, persent_pilU_s, n_b, n_d, n_s, n_U, diff_pilU, fluo_diff_pilU = persentage_difference(df_pilU,0)
print("PilU")
print("Percent of cells with more pili at bright pole = "+str(persent_pilU_b))
print("Percent of cells with more pili at dim pole = "+str(persent_pilU_d))
print("Percent of cells with same number of pili at both poles = "+str(persent_pilU_s))
print('N cells = '+str(n_U)+',  N more at bright pole = '+str(n_b)+',  N more at dim pole = '+str(n_d)+',  N same at both pole = '+str(n_s)+'\n')

persent_pilG_b, persent_pilG_d, persent_pilG_s, n_b, n_d, n_s, n_G, diff_pilG, fluo_diff_pilG = persentage_difference(df_pilG,0)
print("PilG")
print("Percent of cells with more pili at bright pole = "+str(persent_pilG_b))
print("Percent of cells with more pili at dim pole = "+str(persent_pilG_d))
print("Percent of cells with same number of pili at both poles = "+str(persent_pilG_s))
print('N cells = '+str(n_G)+',  N more at bright pole = '+str(n_b)+',  N more at dim pole = '+str(n_d)+',  N same at both pole = '+str(n_s)+'\n')

piliDim=np.array([persent_pilB_d, persent_pilT_d, persent_pilU_d, persent_pilG_d])
piliSame=np.array([persent_pilB_s, persent_pilT_s, persent_pilU_s, persent_pilG_s])
piliBright=np.array([persent_pilB_b, persent_pilT_b, persent_pilU_b, persent_pilG_b])
nb_cells=np.array([n_B, n_T, n_U, n_G])
X=["pilB_mNG", "mNG_pilT", "mNG_pilU", "mNG_pilG"]

PilB
Percent of cells with more pili at bright pole = 0.66
Percent of cells with more pili at dim pole = 0.235
Percent of cells with same number of pili at both poles = 0.105
N cells = 200,  N more at bright pole = 132,  N more at dim pole = 47,  N same at both pole = 21

PilT
Percent of cells with more pili at bright pole = 0.4329896907216495
Percent of cells with more pili at dim pole = 0.41237113402061853
Percent of cells with same number of pili at both poles = 0.15463917525773196
N cells = 194,  N more at bright pole = 84,  N more at dim pole = 80,  N same at both pole = 30

PilU
Percent of cells with more pili at bright pole = 0.4841628959276018
Percent of cells with more pili at dim pole = 0.3574660633484163
Percent of cells with same number of pili at both poles = 0.1583710407239819
N cells = 221,  N more at bright pole = 107,  N more at dim pole = 79,  N same at both pole = 35

PilG
Percent of cells with more pili at bright pole = 0.6538461538461539
Percent of cells with more 

In [13]:
output_file("dodged_bars.html")

data = {'Strain' : X,
        'bright'   : piliBright,
        'dim'   : piliDim,
        'same'   : piliSame}

source = ColumnDataSource(data=data)

p = figure(x_range=X, y_range=(0, 1.1), plot_height=400, plot_width=450, title="Fraction of cells with highest pili number location")

p.vbar(x=dodge('Strain', -0.25, range=p.x_range), top='bright', width=0.2, source=source,
       color="gainsboro", legend_label='Bright')

p.vbar(x=dodge('Strain',  0.0,  range=p.x_range), top='dim', width=0.2, source=source,
       color="darkgrey", legend_label="Dim")

p.vbar(x=dodge('Strain',  0.25, range=p.x_range), top='same', width=0.2, source=source,
       color="black", legend_label="Same")

p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

p.output_backend = 'svg'
bokeh.io.show(p)

In [14]:
Bipolar_pilB=persentage_bipolar_pili(df_pilB)
print("PilB")
print("Percentage of cells with bipolar pili = "+str(Bipolar_pilB))

Bipolar_pilT=persentage_bipolar_pili(df_pilT)
print("PilT")
print("Percentage of cells with bipolar pili = "+str(Bipolar_pilT))

Bipolar_pilU=persentage_bipolar_pili(df_pilU)
print("PilU")
print("Percentage of cells with bipolar pili = "+str(Bipolar_pilU))

Bipolar_pilG=persentage_bipolar_pili(df_pilG)
print("PilG")
print("Percentage of cells with bipolar pili = "+str(Bipolar_pilG))

PilB
Percentage of cells with bipolar pili = 0.33
PilT
Percentage of cells with bipolar pili = 0.5103092783505154
PilU
Percentage of cells with bipolar pili = 0.5520361990950227
PilG
Percentage of cells with bipolar pili = 0.34615384615384615


In [16]:
persent_pilB_b, persent_pilB_d, persent_pilB_s, n_b, n_d, n_s, n_B = persentage_difference_biorep(df_pilB,0)
print("PilB")
print("Percent of cells with more pili at bright pole = "+str(persent_pilB_b))
print("Percent of cells with more pili at dim pole = "+str(persent_pilB_d))
print("Percent of cells with same number of pili at both poles = "+str(persent_pilB_s))
print('N cells = '+str(n_B)+',  N more at bright pole = '+str(n_b)+',  N more at dim pole = '+str(n_d)+',  N same at both pole = '+str(n_s)+'\n')

persent_pilT_b, persent_pilT_d, persent_pilT_s, n_b, n_d, n_s, n_T = persentage_difference_biorep(df_pilT,0)
print("PilT")
print("Percent of cells with more pili at bright pole = "+str(persent_pilT_b))
print("Percent of cells with more pili at dim pole = "+str(persent_pilT_d))
print("Percent of cells with same number of pili at both poles = "+str(persent_pilT_s))
print('N cells = '+str(n_T)+',  N more at bright pole = '+str(n_b)+',  N more at dim pole = '+str(n_d)+',  N same at both pole = '+str(n_s)+'\n')

persent_pilU_b, persent_pilU_d, persent_pilU_s, n_b, n_d, n_s, n_U = persentage_difference_biorep(df_pilU,0)
print("PilU")
print("Percent of cells with more pili at bright pole = "+str(persent_pilU_b))
print("Percent of cells with more pili at dim pole = "+str(persent_pilU_d))
print("Percent of cells with same number of pili at both poles = "+str(persent_pilU_s))
print('N cells = '+str(n_U)+',  N more at bright pole = '+str(n_b)+',  N more at dim pole = '+str(n_d)+',  N same at both pole = '+str(n_s)+'\n')

persent_pilG_b, persent_pilG_d, persent_pilG_s, n_b, n_d, n_s, n_G = persentage_difference_biorep(df_pilG,0)
print("PilG")
print("Percent of cells with more pili at bright pole = "+str(persent_pilG_b))
print("Percent of cells with more pili at dim pole = "+str(persent_pilG_d))
print("Percent of cells with same number of pili at both poles = "+str(persent_pilG_s))
print('N cells = '+str(n_G)+',  N more at bright pole = '+str(n_b)+',  N more at dim pole = '+str(n_d)+',  N same at both pole = '+str(n_s)+'\n')

piliDim=np.array([np.mean(persent_pilB_d), np.mean(persent_pilT_d), np.mean(persent_pilU_d), np.mean(persent_pilG_d)])
piliSame=np.array([np.mean(persent_pilB_s), np.mean(persent_pilT_s), np.mean(persent_pilU_s), np.mean(persent_pilG_s)])
piliBright=np.array([np.mean(persent_pilB_b), np.mean(persent_pilT_b), np.mean(persent_pilU_b), np.mean(persent_pilG_b)])
piliDim_std=np.array([np.std(persent_pilB_d), np.std(persent_pilT_d), np.std(persent_pilU_d), np.std(persent_pilG_d)])
piliSame_std=np.array([np.std(persent_pilB_s), np.std(persent_pilT_s), np.std(persent_pilU_s), np.std(persent_pilG_s)])
piliBright_std=np.array([np.std(persent_pilB_b), np.std(persent_pilT_b), np.std(persent_pilU_b), np.std(persent_pilG_b)])
persent_piliBright=[persent_pilB_b, persent_pilT_b, persent_pilU_b, persent_pilG_b]
persent_piliDim=[persent_pilB_d, persent_pilT_d, persent_pilU_d, persent_pilG_d]
persent_piliSame=[persent_pilB_s, persent_pilT_s, persent_pilU_s, persent_pilG_s]
nb_cells=np.array([n_B, n_T, n_U, n_G], dtype=object)
X=["pilB_mNG", "mNG_pilT", "mNG_pilU", "mNG_pilG"]

PilB
Percent of cells with more pili at bright pole = [0.6607142857142857, 0.6285714285714286, 0.75, 0.7291666666666666, 0.6190476190476191, 0.5]
Percent of cells with more pili at dim pole = [0.2857142857142857, 0.2857142857142857, 0.1, 0.1875, 0.38095238095238093, 0.1]
Percent of cells with same number of pili at both poles = [0.05357142857142857, 0.08571428571428572, 0.15, 0.08333333333333333, 0.0, 0.4]
N cells = [56, 35, 20, 48, 21, 20],  N more at bright pole = [37, 22, 15, 35, 13, 10],  N more at dim pole = [16, 10, 2, 9, 8, 2],  N same at both pole = [3, 3, 3, 4, 0, 8]

PilT
Percent of cells with more pili at bright pole = [0.46938775510204084, 0.54, 0.4411764705882353, 0.3103448275862069, 0.3125]
Percent of cells with more pili at dim pole = [0.4897959183673469, 0.36, 0.38235294117647056, 0.3793103448275862, 0.4375]
Percent of cells with same number of pili at both poles = [0.04081632653061224, 0.1, 0.17647058823529413, 0.3103448275862069, 0.25]
N cells = [49, 50, 34, 29, 32], 

In [17]:
output_file("dodged_bars.html")

data = {'Strain' : X,
        'bright'   : piliBright,
        'dim'   : piliDim,
        'same'   : piliSame}

source = ColumnDataSource(data=data)

p = figure(x_range=X, y_range=(0, 1.1), plot_height=400, plot_width=450, title="Fraction of cells with highest pili number location")

p.vbar(x=dodge('Strain', -0.25, range=p.x_range), top='bright', width=0.2, source=source,
       color="gainsboro", legend_label='Bright')

p.vbar(x=dodge('Strain',  0.0,  range=p.x_range), top='dim', width=0.2, source=source,
       color="darkgrey", legend_label="Dim")

p.vbar(x=dodge('Strain',  0.25, range=p.x_range), top='same', width=0.2, source=source,
       color="black", legend_label="Same")

p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"


errobar(piliBright, piliBright_std, p, 0.25)
errobar(piliDim, piliDim_std, p, 0.5)
errobar(piliSame, piliSame_std, p, 0.75)


p.output_backend = 'svg'
bokeh.io.show(p)

In [9]:
p = figure(x_range=X, y_range=(0, 1), plot_height=400, plot_width=450, title="Fraction of cells with highest pili number location")
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

plot_data(persent_piliBright, p, 0.25, "firebrick", 0.01, 'Bright')
errobar(piliBright, piliBright_std, p, 0.25)
plot_mean(piliBright, p, 0.25)

plot_data(persent_piliDim, p, 0.5, "yellow", 0.01, 'Dim')
errobar(piliDim, piliDim_std, p, 0.5)
plot_mean(piliDim, p, 0.5)

plot_data(persent_piliSame, p, 0.75, "black", 0.01, 'Same')
errobar(piliSame, piliSame_std, p, 0.75)
plot_mean(piliSame, p, 0.75)

p.output_backend = 'svg'
bokeh.io.show(p)

C:\ProgramData\Anaconda3\lib\site-packages\bokeh\models\plots.py:766: UserWarning: 
You are attempting to set `plot.legend.location` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)
C:\ProgramData\Anaconda3\lib\site-packages\bokeh\models\plots.py:766: UserWarning: 
You are attempting to set `plot.legend.orientation` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)
